# MQL testing

This notebook is for performance testing of MQL queries.

# Preliminaries

MQL queries are performed by the Emdros software, which is written in C++.

We use the Python bindings (SWIG) to steer the Emdros engine.

These Python bindings need Python2.

In order to have a Python2 kernel in Jupyter under an Anaconda installation that is based on Python3, I had to perform the following sequence:

    conda create -n py27 python=2.7
    source activate py27
    conda install notebook ipykernel
    sudo ipython kernel install
    source deactivate

# Problem queries

MQL queries may easily deliver billions of results. 
Usually, it is not the intention of the user to get that many results.
Most of the time, it is caused by limited experience with the MQL language.

Here is an example query (see also 
[this query on SHEBANQ](https://shebanq.ancient-data.org/hebrew/query?version=4b&id=1119)
):

    select all objects where
        [word focus lex = 'R>CJT/']
        ..
        [word]
        ..
        [word lex = 'BR>[']

The problem is that every combination of three words in the 425000 words in the Bible of which the first and third meet the indicated requirements counts as a result.
So, if we have say 10 candidates for the first word in Genesis, and 10 candidates for the second in Chronicles, then we have 100 combinations. And every such combination has at least 300000 words in between, so we get 30 million results.

This kind of problem can be remedied by putting the word blocks in a container, e.g.

    select all objects where
        [chapter
            [word focus lex = 'R>CJT/']
            ..
            [word]
            ..
            [word lex = 'BR>[']
        ]
        
and most of the time this corresponds better with the intention of the query writer.
Instead of a wild multiplication of search results, we have a more or less linear behaviour of the amount of search results.

# Performance testing

We are going to conduct several experiments with the "bad" query above, just to collect some statistics.

The resulting insights have been applied in the ``mql.py`` module in the SHEBANQ web application.

In [1]:
import sys, time, collections
from functools import reduce
from operator import mul
from operator import add

sys.path.append('/opt/emdros/lib/emdros')
sys.path.append('/Users/dirk/SURFdrive/current/demos/github/shebanq/modules')
import EmdrosPy
from get_db_config import config, emdros_versions

db='shebanq_etcbc'

OLD_EMDROS_VERSIONS = set(emdros_versions[0:-1])
EMDROS_VERSION = emdros_versions[-1]

timestamp = None

def _reset():
    global timestamp
    timestamp = time.time()

def _elapsed():
    interval = time.time() - timestamp
    if interval < 10: return "{: 2.2f}s".format(interval)
    interval = int(round(interval))
    if interval < 60: return "{:>2d}s".format(interval)
    if interval < 3600: return "{:>2d}m {:>02d}s".format(interval // 60, interval % 60)
    return "{:>2d}h {:>02d}m {:>02d}s".format(interval // 3600, (interval % 3600) // 60, interval % 60)

def msg(msg, newline=True, withtime=True):
    timed_msg = "{:>7} ".format(_elapsed()) if withtime else ''
    timed_msg += msg
    if newline: timed_msg += "\n"
    sys.stderr.write(timed_msg)
    sys.stderr.flush()

def Msg(txt, newline=True, withtime=True):
    _reset()
    msg(txt, newline=newline, withtime=withtime)

def sanitize(query, msgs):
    comps = query.split('/*')
    lastcomp = comps[-1]
    if len(comps) > 1 and lastcomp.find('*/') == -1:
        result = query + '*/'
    else:
        result = query
    if 'focus' not in query.lower():
        msgs.append(('note', 'no FOCUS in your query!'))
    return result + "\nGO\n"

def to_monadsets(setstr):
    elems = setstr[2:-2].strip()
    if elems == '': return []
    comps = elems.split(',')
    return [[int(y) for y in x.lstrip().split('-')] if '-' in x else [int(x), int(x)] for x in comps]


# Test queries

We will test with the above "bad" query.
We will also use test versions that restrict the results to certain initial stretches of the total text, say 100,000 words, 200,000 and 300,000.

In [2]:
query_tpl = '''
select all objects {} where
    [word focus lex = 'R>CJT/']
    ..
    [word]
    ..
    [word lex = 'BR>[']
'''

queries = collections.OrderedDict((
    ('r10', query_tpl.format('in {1-100000}')),
    ('r20', query_tpl.format('in {1-200000}')),
    ('r25', query_tpl.format('in {1-250000}')),
    ('all', query_tpl.format('')),
))

# MQL results: the Sheaf

The sheaf is a sophisticted data structure that stores the results of an MQL query.
In general, the results of one and the same MQL query share a considerable amount of information. The sheaf fully utilizes and reflects this data sharing.

The flip side is, that in order to get the concrete results, we have to recursively walk this data structure and construct results.

Even when we start counting the results, we have to make this walk, and this can be rather costly.

The sheaf is a C++ datastructure, delivered by the Emdros software.

In our case, with this concrete problem query, the sheaf has ~130,000,000 results.
It takes Emdros roughly 5 minutes to build it up. There is very little sql time involved, the bulk of the time is C processing.

We hit a performance penalty when we use a scripting language to walk this structure.
Even if we use fast and efficient Python constructs such as iterators and generators, it takes much more time than it took Emdros to create the sheaf.

It turns out that it takes nearly half an hour to just count the number of results.

That is a pity, since the delivering of the results that SHEBANQ needs, goes much faster.
SHEBANQ does not need the list of results, but the set of monads that occur in a result.

Emdros has an API function to deliver exactly this, so we can the big monad set with C speed. 
And this is never bigger than the set of all monads, which has a size of 425000.

So the mere counting of results causes a 3 fold increase in waiting time.

We have to do something about it.

## Optimizing the Python code

It turns out that using the generator idiom and not the for-loop-idiom does not help here.
Problably the performance hit is in SWIG, which maps slick C++-data structures to Python data structures all the time. We cannot change that machinery.

## Constraining the sheaf

When we are counting, we can decide to stop counting as soon as we have reached a certain number of results. This will dramatically cut down the counting effort, at the expense of not being able to count the number of results of offending queries.

We opt for a limit that is slightly larger than the number of words in the bible, and we hope that this will not constrain people. We believe that offending queries are mostly a result of unlucky attempts to formalize a problem into an MQL query.
If that is true, it is better to be able to notify the user and not to waste valuable server time.

In [3]:
ITER_LIMIT = 500000

class LimitError(Exception):
    def __init__(self, message, cause=None):
        Exception.__init__(self, message)

def elements_of(xiter):
    i = 0
    while xiter.hasNext():
        i += 1
        if i > ITER_LIMIT:
            raise LimitError('')
            break
        yield xiter.current()
        xiter.next()
        
limits = (
    100000,
    500000,
    1000000,
    1000000000,
)

# For loop idiom and Generator idiom

In the for loop idiom we use straightforward code based on for loops.

In the Generator idiom we use generators and the function ``reduce``.

In [4]:
def sheaf_results_f(sheaf):
    sh_iter = sheaf.const_iterator()
    n = 0
    while sh_iter.hasNext():
        straw = sh_iter.current()
        n += straw_results_f(straw)
        if n > ITER_LIMIT:
            raise LimitError('')
        sh_iter.next()
    return n

def straw_results_f(straw):
    n = 1
    st_iter = straw.const_iterator()
    while st_iter.hasNext():
        mo = st_iter.current()
        if not mo.sheafIsEmpty():
            sheaf = mo.getSheaf()
            n *= sheaf_results_f(sheaf)
            if n > ITER_LIMIT:
                raise LimitError('')
        st_iter.next()
    return n

def sheaf_results_g(sheaf):
    try:
        n = reduce(
            lambda x,y: add(x, straw_results_g(y)), 
            elements_of(sheaf.const_iterator()), 
            0,
        ) 
    except LimitError as e:
        n = ITER_LIMIT
        raise LimitError('')
    return n

def straw_results_g(straw):
    try:
        n = reduce(
            lambda x,y: mul(x, sheaf_results_g(y.getSheaf()) if not y.sheafIsEmpty() else 1),
            elements_of(straw.const_iterator()),
            1,
        ) 
    except LimitError as e:
        n = ITER_LIMIT
        raise LimitError('')
    return n

idioms = dict(
    f=(sheaf_results_f, straw_results_f),
    g=(sheaf_results_g, straw_results_g),

)

idiom_names = dict(
    f='For',
    g='Gen',
)

def set_idiom(i):
    global sheaf_results
    global straw_results
    (sheaf_results, straw_results) = idioms[i]

# Common code

Here is code used by both idioms

In [5]:
def get_sheaf(vr, query):
    env = EmdrosPy.EmdrosEnv(EmdrosPy.kOKConsole, EmdrosPy.kCSUTF8, config['shebanq_host'], config['shebanq_user'], config['shebanq_passwd'], db+vr, EmdrosPy.kMySQL)
    compiler_result = False
    msgs = []

    msg('{:<10}: fetching the sheaf'.format('C++'))
    good = env.executeString(sanitize(query, msgs) , compiler_result, False, False)[1]
    sheaf = env.getSheaf() if good and env.isSheaf else None
    msg('{:<10}: fetching done{}'.format('C++', ' NO SHEAF!' if sheaf == None else ''))
    # we only need the sheaf, but if we do not return the env, it will be garbage collected
    # and then we get a segmentation fault
    return (env, sheaf)

def get_ms(sheaf):                
    msg('{:<10}: making monadsets'.format('Python'))
    ms = to_monadsets(sheaf.getSOM(True).toString())
    msg('{:<10}: returning {} monadsets'.format('Python', len(ms)))
    return ms

def get_n(sheaf):
    msg('{:<10}: computing number of results'.format('SWIG'))
    limit_exceeded = False
    try:
        n = sheaf_results(sheaf)
    except LimitError as e:
        n = ITER_LIMIT
        limit_exceeded = True
    msg('{:<10}: {} results{})'.format('SWIG', n, 'exceeded' if limit_exceeded else ''))

# Experiments

Here we show the results of executing various options.

In [ ]:
data_version = '4'

def do_all():
    global ITER_LIMIT
    for q in queries:
        Msg('QUERY {}'.format(q))
        query = queries[q]
        (env, sheaf) = get_sheaf(data_version, query)
        if sheaf == None: continue
        ms = get_ms(sheaf)
        for lm in limits:
            Msg('LIMIT {}'.format(lm))
            ITER_LIMIT = lm
            for i in idioms:
                Msg('IDIOM {}'.format(idiom_names[i]))
                set_idiom(i)
                get_n(sheaf)

In [ ]:
do_all()

  0.00s QUERY r10
  0.01s C++       : fetching the sheaf
  4.16s C++       : fetching done
  4.17s Python    : making monadsets
  4.28s Python    : returning 11 monadsets
  0.00s LIMIT 100000
  0.00s IDIOM Gen
  0.00s SWIG      : computing number of results
  1.12s SWIG      : 100000 resultsexceeded)
  0.00s IDIOM For
  0.00s SWIG      : computing number of results
  1.11s SWIG      : 100000 resultsexceeded)
  0.00s LIMIT 500000
  0.00s IDIOM Gen
  0.00s SWIG      : computing number of results
  5.85s SWIG      : 500000 resultsexceeded)
  0.00s IDIOM For
  0.00s SWIG      : computing number of results
  4.73s SWIG      : 500000 resultsexceeded)
  0.00s LIMIT 1000000
  0.00s IDIOM Gen
  0.00s SWIG      : computing number of results
    11s SWIG      : 942880 results)
  0.00s IDIOM For
  0.00s SWIG      : computing number of results
  9.33s SWIG      : 942880 results)
  0.00s LIMIT 1000000000
  0.00s IDIOM Gen
  0.00s SWIG      : computing number of results
    11s SWIG      : 942880 res